In [1]:
pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain-nomic

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_ollama import ChatOllama
import os
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
import json
from langchain_core.messages import HumanMessage, SystemMessage

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
local_llm = "llama3.2:1b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")


#we use Tavily, which is a search engine optimized for LLMs and RAG
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("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")
os.environ["TOKENIZERS_PARALLELISM"] = "true"


urls = [
    "https://npousa.com/",
    "https://npousa.com/solutions-services/",
    "https://npousa.com/our-approach/",
    "https://npousa.com/customers/",
    "https://npousa.com/contact-us/?_gl=1*4cb6u6*_up*MQ..*_ga*NjcwMDk5MjY3LjE3Mjc4NDExMDk.*_ga_J5P9LNHH6S*MTcyNzg0MTEwOC4xLjAuMTcyNzg0MTEwOC4wLjAuMA.."
]

# Load documents
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

# Create retriever
retriever = vectorstore.as_retriever(k=3)

import json
from langchain_core.messages import HumanMessage, SystemMessage





In [10]:
### Generate
question = "What does NPO USA offer in cloud solutions?"
# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

NPO USA offers a comprehensive suite of cloud solutions, including financial operations optimization (FinOps), DevOps strategies, cloud consulting (Cloud Advisor), seamless cloud migration journeys (Journey to Cloud), sophisticated cloud architecture design, advanced cloud automation, and efficient multi-cloud management. These services are designed to empower businesses across the globe with agility, scalability, and cost reduction. NPO USA's cloud solutions cater to various demands, providing a tailored approach that meets clients' needs in a consultative manner.


In [4]:
import time
from datetime import datetime, timedelta

# Placeholder function to retrieve documents based on a question (use actual retriever logic)
def retrieve_docs(question):
    docs = retriever.invoke(question)  # Modify this based on your retrieval method
    return format_docs(docs)

# Formatting function to prepare context from documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt structure
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer: """

# Function to handle a single question-answer cycle
def get_answer(question, context=""):
    docs_txt = retrieve_docs(question)
    updated_context = context + "\n\n" + docs_txt if context else docs_txt
    rag_prompt_formatted = rag_prompt.format(context=updated_context, question=question)
    generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
    return generation.content, updated_context

# Main session loop
def qa_session(session_duration=300):
    context = ""
    start_time = datetime.now()
    session_end_time = start_time + timedelta(seconds=session_duration)
    
    print(f"Session started. You have {session_duration // 60} minutes.")

    while datetime.now() < session_end_time:
        question = input("\nAsk a question: ")
        if not question.strip():
            print("Ending session as no question was provided.")
            break

        answer, context = get_answer(question, context)
        print("\nAnswer:", answer)

    print("Session ended.")

# Running a 5-minute QA session
qa_session(300)


Session started. You have 5 minutes.

Answer: I am an assistant for question-answering tasks, providing information and answering inquiries based on the provided context.


KeyboardInterrupt: Interrupted by user

In [10]:
### Hallucination Grader

# Hallucination grader instructions
hallucination_bot_instructions = """

You are a Chatbot assistant who provides information to NPO USA Inc Website Visiters. 
You should not asssume anything.
Make sure to give only information available on website.
Do not add any extra information
"""

# Grader prompt
hallucination_bot_prompt = """FACTS: \n\n {documents} \n\n STUDENT ANSWER: {generation}. 

"""
# Test using documents and generation from above
docs_txt = retrieve_docs(question)
hallucination_grader_prompt_formatted = hallucination_bot_prompt.format(
    documents=docs_txt, generation=generation.content
)
result = llm_json_mode.invoke(
    [SystemMessage(content=hallucination_bot_instructions)]
    + [HumanMessage(content=hallucination_grader_prompt_formatted)]
)
json.loads(result.content)

NameError: name 'docs_txt' is not defined